# 模型部署

In [1]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
CHECKPOINT_PATH = "./../output/chatglm-6b-pt-128-2e-2/checkpoint-3000"

# 载入Tokenizer
tokenizer = AutoTokenizer.from_pretrained("./chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [ ]:
config = AutoConfig.from_pretrained("./chatglm-6b", trust_remote_code=True, pre_seq_len=128)
model = AutoModel.from_pretrained("./chatglm-6b", config=config, trust_remote_code=True)
prefix_state_dict = torch.load(os.path.join(CHECKPOINT_PATH, "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

# 设置模型为评估模式
model = model.half().cuda()  # 转换为半精度并移至GPU
model = model.eval()  # 设置为评估模式

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/lick/tools/anaconda3/envs/TCMLLM/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/lick/tools/anaconda3/envs/TCMLLM/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/lick/tools/anaconda3/envs/TCMLLM/lib/python3.10/site-packages/accelerate/utils/torch_xla.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at ./chatglm-6b and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
# 交互式聊天函数
def chat():
    print("开始与模型对话（输入'stop'退出，'clear'清空历史记录）")
    history = []
    while True:
        query = input("\n用户：")
        if query.strip() == "stop":
            break
        if query.strip() == "clear":
            history = []
            print("已清空历史记录")
            continue
            
        print(f"\n用户：{query}")  # 显示用户输入
        response, history = model.chat(tokenizer, query, history=history)
        print(f"助手：{response}")

# 开始对话
chat()

开始与模型对话（输入'stop'退出，'clear'清空历史记录）



用户：你好
助手：你好！请问有什么需要帮助的吗？

用户：你是基于哪个LLM进行微调的？
助手：我是基于清华大学 KEG 实验室和智谱AI共同训练的语言模型进行微调的。

用户：现在你擅长什么任务？
助手：作为一个语言模型，我擅长执行各种语言任务，包括文本分类、情感分析、机器翻译、语音识别等。

用户：翻译：我感觉很烦躁，但是不发烧也不感冒。请帮我推荐中药或者方剂。
助手：根据您提供的信息，您可能正在经历情绪上的烦躁。以下是一些中药或方剂的推荐：

1. 逍遥散：逍遥散是一种常用的中药方剂，适用于情绪烦躁、失眠、头痛等症状。它由柴胡、黄芩、半夏、人参、甘草等药材组成。

2. 丹参酮胶囊：丹参酮胶囊是一种中药成分，有助于缓解情绪上的烦躁和焦虑。它由丹参、桃仁、红花、川芎等药材组成。

3. 归脾丸：归脾丸是一种中药方剂，适用于情绪烦躁、失眠、食欲不振等症状。它由人参、黄芪、白术、当归、茯苓、甘草等药材组成。

请注意，这些中药或方剂应在医生的指导下使用，以确保安全和有效性。

用户：翻译成英文：我感觉很烦躁，但是不发烧也不感冒。请帮我推荐中药或者方剂。
助手：Based on your given information, it seems that you are experiencing emotional烦躁. Here are some suggestions for中药 or方剂：

1. 柴胡汤：柴胡汤是一种常用的中药方剂，适用于情绪烦躁、失眠、头痛等症状。它由柴胡、黄芩、半夏、人参、甘草等药材组成。

2. 丹参酮胶囊：丹参酮胶囊是一种中药成分，有助于缓解情绪上的烦躁和焦虑。它由丹参、桃仁、红花、川芎等药材组成。

3. 归脾丸：归脾丸是一种中药方剂，适用于情绪烦躁、失眠、食欲不振等症状。它由人参、黄芪、白术、当归、茯苓、甘草等药材组成。

Please note that these中药 or方剂 should be used under the guidance of a doctor to ensure safety and effectiveness.

用户：请问，龟鹤药膏对于清热解毒有什么作用？
助手：龟鹤药膏是一种传统的中药材，常用于清热解毒。它的主要成分包括连翘、板蓝根、野菊花、薄荷等，具有清热解毒、抗病毒、抗菌、消炎等功效。

龟鹤